In [11]:
using LinearAlgebra
using Distributed
using SharedArrays
using BenchmarkTools
using Printf

In [3]:
# Functions 
# System Equation Linear Sparse
function system_matrix_sparse(systemDimension::Integer, matrixA::AbstractMatrix)
    vectorDiagonalMain = 3 * ones(systemDimension)
    vectorDiagonalUpperLower = -1 * ones(systemDimension - 1)
    matrixA .= convert(Matrix, Tridiagonal(vectorDiagonalUpperLower, vectorDiagonalMain, vectorDiagonalUpperLower))
    for i = 1:systemDimension
        if matrixA[i, systemDimension-i+1] == 0.0
            matrixA[i, systemDimension-i+1] = 0.5
        end
    end
    return nothing
end
# 
function vector_independent_term(systemDimension::Integer, vectorB::AbstractVector)
    vectorB[1] = 2.5
    vectorB[systemDimension] = 2.5
    position::Integer = floor(systemDimension / 2)
    for i = 2:systemDimension-1
        if i == position || i == position + 1
            vectorB[i] = 1.0
        else
            vectorB[i] = 1.5
        end
    end
    return nothing
end

vector_independent_term (generic function with 1 method)

In [4]:
n = 20
A = Matrix{Float64}(undef, n, n)
b = Vector{Float64}(undef, n)
system_matrix_sparse(n,A)
vector_independent_term(n, b)
ϵ = 0.01
δ = 0.001;

In [5]:
M = diagm(0 => diag(A));
N = M-A;
T = inv(M) * N;
f = inv(M) * b;
nT, mT = size(T);
Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1;

In [6]:
#  Monte Carlo Markov Chain Method 

function mcmc_acc_par_ta(ϵ, δ,A::Matrix{Float64}, b::Vector{Float64})
    M = diagm(0 => diag(A))
    N = M-A
    T = inv(M) * N
    f = inv(M) * b
    mT, nT = size(T)
    display(size(T))
    S = fill(0, nT)
    P = fill(0., mT, mT) 
    [S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
    [P[i,j]= 1/S[i] for i in 1:mT, j in 1:mT if T[i,j] != 0 ]
    Pa = [accumulate(+, P[i, 1:mT]) for i in 1:mT]
    #Pi = [1/mT for i in 1:mT]
    Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1
    
    Xs = fill(0., mT)
    for i in 1:mT
        W_0 = 1.0
        for s in 1:Nc
            W = W_0; point = i; X = W_0 * f[i]
            while abs(W) >= ϵ
                nextpoint  = 1
                u = rand()
                while u >= Pa[point][nextpoint]
                    nextpoint = nextpoint + 1
                end
                if T[point, nextpoint] != 0 
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])
                    X = X + W_new * f[nextpoint]
                end
                point = nextpoint
                W = W_new
            end
        Xs[i] += X
        end
    end
    Xs = Xs/Nc;
end

mcmc_acc_par_ta (generic function with 1 method)

In [7]:
Xs = mcmc_acc_par_ta(ϵ, δ, A, b)


(20, 20)

20-element Vector{Float64}:
 1.0003082972156194
 1.001114481137491
 1.0003512505697734
 1.0018870317889415
 1.0006121339836902
 1.0028788548507088
 0.9996181310478135
 1.0032554248943892
 1.0007759918980246
 0.9995908782225991
 1.0002861925127793
 1.0003320198233896
 1.0008037625588706
 1.0030174600146566
 1.0019091760187655
 1.0012150375889528
 1.0011052822106843
 1.001034100836993
 1.0021221402949594
 0.999969096328459

In [8]:
error = norm(b-A*Xs)


0.01905187177883472

In [9]:
BenchmarkTools.DEFAULT_PARAMETERS.samples = 3
Xs = mcmc_acc_par_ta(ϵ, δ, A, b);
error = norm(b-A*Xs)
time = @benchmark mcmc_acc_par_ta($ϵ, $δ, $A, $b)
elapsed_time = mean(time.times)/1000000000
#println(error, ", ", elapsed_time)
@printf("%.6f, %.6f, %d\n", error, elapsed_time, Nc) 

0.013776, 13.115165, 1855098


In [25]:
# Add worker processes
addprocs(4)  # Adjust the number of processes as needed

@everywhere begin
    using Random
    using LinearAlgebra
    using SharedArrays
end

In [29]:
@everywhere function mcmc_iteration!(Xs, ϵ, δ, A, b)
    M = diagm(0 => diag(A))
    N = M - A
    T = inv(M) * N
    f = inv(M) * b
    mT, nT = size(T)
    S = zeros(Int, nT)
    P = zeros(Float64, mT, mT)
    for i in 1:mT, j in 1:mT
        if T[i, j] != 0
            S[j] += 1
            P[i, j] = 1 / S[j]
        end
    end
    Pa = [cumsum(P[i, :]) for i in 1:mT]
    Nc = Int(floor((0.6745 / δ)^2 * ((norm(f)^2) / (1 - norm(T))^2)) + 1)

    local_Xs = zeros(Float64, mT)
    for i in 1:mT
        for s in 1:Nc
            W = 1.0
            point = i
            X = W * f[i]
            while abs(W) >= ϵ
                nextpoint = 1
                u = rand()
                while u >= Pa[point][nextpoint]
                    nextpoint += 1
                end
                if T[point, nextpoint] != 0
                    W_new = W * (T[point, nextpoint] / P[point, nextpoint])
                    X += W_new * f[nextpoint]
                end
                point = nextpoint
                W = W_new
            end
            local_Xs[i] += X
        end
    end
    return local_Xs ./ Nc
end

function parallel_mcmc(ϵ, δ, A, b, num_iterations)
    nworkers() > 1 || error("Parallel computing requires more than one worker.")

    Xs = SharedArray{Float64}(zeros(length(b)))
    @distributed for _ in 1:num_iterations
        Xs .+= mcmc_iteration!(zeros(length(b)), ϵ, δ, A, b)
        display(Xs)
    end

    return Xs ./ num_iterations
end


Unhandled Task ERROR: On worker 10:
BoundsError: attempt to access 20-element Vector{Float64} at index [21]
Stacktrace:
  [1] getindex
    @ ./essentials.jl:13 [inlined]
  [2] mcmc_iteration!
    @ ~/Documentos/WorkSpaces/pesadilla/mcmc.ipynb:27
  [3] macro expansion
    @ ~/Documentos/WorkSpaces/pesadilla/mcmc.ipynb:48 [inlined]
  [4] #53
    @ ~/.julia/juliaup/julia-1.10.2+0.x64.linux.gnu/share/julia/stdlib/v1.10/Distributed/src/macros.jl:303
  [5] #178
    @ ~/.julia/juliaup/julia-1.10.2+0.x64.linux.gnu/share/julia/stdlib/v1.10/Distributed/src/macros.jl:83
  [6] #invokelatest#2
    @ ./essentials.jl:892 [inlined]
  [7] invokelatest
    @ ./essentials.jl:889
  [8] #107
    @ ~/.julia/juliaup/julia-1.10.2+0.x64.linux.gnu/share/julia/stdlib/v1.10/Distributed/src/process_messages.jl:283
  [9] run_work_thunk
    @ ~/.julia/juliaup/julia-1.10.2+0.x64.linux.gnu/share/julia/stdlib/v1.10/Distributed/src/process_messages.jl:70
 [10] run_work_thunk
    @ ~/.julia/juliaup/julia-1.10.2+0.x64.lin

parallel_mcmc (generic function with 1 method)

In [28]:
parallel_mcmc(ϵ, δ, A, b, 40);